In [1]:
import os, sys
import scanpy as sc
import pandas as pd
import numpy as np
import json
import warnings
warnings.filterwarnings("ignore")

In [2]:
from spamint import sprout_plus
from spamint import preprocess as pp
#import logging
#logger = logging.getLogger(__name__)
logger = sprout_plus.logger

# 1. Load files

In [3]:
logger.info("Loading files...")
inputDir = './tutorial/SCC_demo/'
outDir = f'{inputDir}/results/'
sc_exp = pd.read_csv(f'{inputDir}/SC_exp.tsv',sep = '\t',header=0,index_col=0)
sc_meta = pd.read_csv(f'{inputDir}/SC_meta.tsv',sep = '\t',header=0,index_col=0)
st_exp = pd.read_csv(f'{inputDir}/ST_exp.tsv',sep = '\t',header=0,index_col=0)
st_coord = pd.read_csv(f'{inputDir}/ST_coord.csv',sep = ',',header=0,index_col=0)
st_decon = pd.read_csv(f'{inputDir}/ST_decon.tsv',sep = '\t',header=0,index_col=0)
sc_smurf = pd.read_csv(f'{inputDir}/smurf_ref.csv',sep = ',',header=0,index_col=0)
lr_df = pd.read_csv('./LR/human_LR_pairs.txt',sep='\t',header=None)
logger.info("File loading complete")

[2024-08-01 14:26:36,528/INFO] Loading files...
[2024-08-01 14:27:58,754/INFO] File loading complete


# Parameters

In [4]:
logger.info("Setting parameters...")
st_decon.columns = [x.split('sf_')[-1] for x in st_decon.columns]
species = 'Human'
st_tp = 'visum'
meta_key = 'level3_celltype'
SUM = 1e4
alpha, beta, gamma, delta, eta = [1, 0.001, 0.001, 0.1, 0.0005]
if st_tp == 'slide-seq':
    num_per_spot = 1
    repeat_penalty = 2
else:
    num_per_spot = 10
    repeat_penalty = int((st_exp.shape[0] * num_per_spot/sc_exp.shape[0]) * 10)

max_rep = 3

[2024-08-01 14:27:58,766/INFO] Setting parameters...


In [30]:
logger.debug("Reloading all SpaMint modules")
import importlib
l = [module for module in sys.modules.values() if module.__name__.startswith('spamint')]
for module in l:
    try:
        if module.__name__.startswith('spamint'):
            print(module.__name__)
            importlib.reload(module)
    except:
        pass

[2024-08-01 15:58:40,572/DEBUG] Reloading all SpaMint modules


spamint
spamint.utils
spamint.optimizers
spamint.preprocess
spamint.cell_selection
spamint.gd_torch
spamint.sprout_plus


# Preprocess

In [6]:
if st_exp.shape[1]<1e4:
    # merfish data, only has 200~500 genes
    sc_adata, st_adata, sc_ref, lr_df = pp.prep_all_adata_merfish(sc_exp = sc_exp, st_exp = st_exp, sc_distribution = sc_smurf, 
                                                        sc_meta = sc_meta, st_coord = st_coord, lr_df = lr_df, SP = species)
else:
    sc_adata, st_adata, sc_ref, lr_df = pp.prep_all_adata(sc_exp = sc_exp, st_exp = st_exp, sc_distribution = sc_smurf, 
                                                            sc_meta = sc_meta, st_coord = st_coord, lr_df = lr_df, SP = species)

[2024-08-01 14:28:08,077/DEBUG] Data clean is done! Using 15386 shared genes .


In [7]:
obj_spex = sprout_plus.SpaMint(save_path = outDir, st_adata = st_adata, weight = st_decon, 
                sc_ref = sc_ref, sc_adata = sc_adata, cell_type_key = meta_key, lr_df = lr_df, 
                st_tp = st_tp)
#obj_spex.prep()

sc_ref and sc_adata has different genes, both data are subset to 15386 genes.
[2024-08-01 14:28:09,015/DEBUG] Parameters checked!
[2024-08-01 14:28:18,449/DEBUG] Getting svg genes
By setting k as 4, each spot has average 3.990990990990991 neighbors.
[2024-08-01 14:28:18,696/DEBUG] Calculating spots affinity profile data
[2024-08-01 14:28:22,211/DEBUG] SpaMint object created.


# cell selection

In [18]:
sc_agg_meta = obj_spex.select_cells(use_sc_orig = True, p = 0, mean_num_per_spot = num_per_spot,
                                    mode = 'strict', max_rep = 1,
                                    repeat_penalty = repeat_penalty)
sc_agg_meta.to_csv(f'{outDir}/spexmod_sc_meta.tsv',sep = '\t',header=True,index=True)

[2024-07-22 08:49:19,169/INFO] Starting cell selection
[2024-07-22 08:49:19,170/DEBUG] 0. calc num of cell per spot
[2024-07-22 08:49:19,172/DEBUG] 	 Estimating the cell number in each spot by the deconvolution result.
[2024-07-22 08:49:21,401/DEBUG] 1. filter gene
[2024-07-22 08:49:21,566/DEBUG] 2. feature select
[2024-07-22 08:49:21,568/DEBUG] 	 SpexMod selects 3430 feature genes.
[2024-07-22 08:49:21,569/DEBUG] 3. scale and norm
[2024-07-22 08:49:25,042/DEBUG] 4. init solution
[2024-07-22 08:50:49,440/DEBUG] 	 Init solution: max - 0.9472,     mean - 0.6388,     min - 0.1398
[2024-07-22 08:50:51,622/DEBUG] 5. Swap selection start...
[2024-07-22 08:50:51,623/DEBUG] 	Swap selection iter 0 of 1
[2024-07-22 08:56:18,272/DEBUG] 	 Swapped solution: max - 0.93,     mean - 0.70,     min - 0.15


if p != 0, extremely time consuming

In [ ]:
# change p to 0.1, use different code to select cells
sc_agg_meta = obj_spex.select_cells(use_sc_orig = True, p = 0.1, mean_num_per_spot = num_per_spot, mode = 'strict', max_rep = 1, 
                                    repeat_penalty = repeat_penalty)
sc_agg_meta.to_csv(f'{outDir}/spexmod_sc_meta.tsv',sep = '\t',header=True,index=True)

use user defined cell selection

In [ ]:
# use our result to pretend the user provided cell selection
# the user_sc_agg_meta should contain celltype, spot, sc_id columns

In [8]:
user_sc_agg_meta = pd.read_csv(f'{outDir}/spexmod_sc_meta.tsv',sep = '\t',header=0,index_col=0)
user_sc_exp = sc_exp.loc[user_sc_agg_meta['sc_id']]
sc_agg_meta = obj_spex.load_predefined_cells(user_sc_exp, user_sc_agg_meta)

In [21]:
alter_sc_exp, sc_agg_meta = obj_spex.gradient_descent(
                alpha, beta, gamma, delta, eta, 
                init_sc_embed = sc_agg_meta[['adj_spex_UMAP1','adj_spex_UMAP2']],
                iteration = max_rep, k = 2, W_HVG = 2,
                left_range = 1, right_range = 2, steps = 1, dim = 2)
sc_agg_meta.to_csv(f'{outDir}/spexmod_sc_meta.tsv',sep = '\t',header=True,index=True)

[2024-07-22 09:02:40,539/DEBUG] Running v12 now...
[2024-07-22 09:02:40,582/DEBUG] Init sc_coord by affinity embedding...
[2024-07-22 09:02:40,584/DEBUG] Start embedding...
[2024-07-22 09:02:40,585/DEBUG] Calc aff mat...
[2024-07-22 09:03:22,035/DEBUG] Preprocessing affinity matrix
[2024-07-22 09:03:47,177/DEBUG] Evaluating coord generated by UMAP...
[2024-07-22 09:03:51,816/DEBUG] shape correlation is: 0.961643958842655
[2024-07-22 09:03:51,830/DEBUG] End embedding.
[2024-07-22 09:04:33,173/DEBUG] First-term calculation done!
[2024-07-22 09:05:35,487/DEBUG] Second-term calculation done!
[2024-07-22 09:09:13,870/DEBUG] Third term calculation done!
[2024-07-22 09:14:01,310/DEBUG] Fourth term calculation done!
[2024-07-22 09:14:02,083/DEBUG] Hyperparameters adjusted.
[2024-07-22 09:14:02,084/DEBUG] -----Start iteration 0 -----
[2024-07-22 09:14:02,085/DEBUG] Start embedding...
[2024-07-22 09:14:02,086/DEBUG] Calc aff mat...
[2024-07-22 09:14:38,219/DEBUG] Preprocessing affinity matrix
[2

# Gradient desent

In [31]:
alter_sc_exp, sc_agg_meta = obj_spex.gradient_descent(
                alpha, beta, gamma, delta, eta, 
                init_sc_embed = False,
                iteration = 1, k = 2, W_HVG = 2,
                left_range = 1, right_range = 2, steps = 1, dim = 2)
sc_agg_meta.to_csv(f'{outDir}/spexmod_sc_meta.tsv',sep = '\t',header=True,index=True)
# with open(f'{outDir}/sc_knn.json', 'w') as fp:
#     json.dump(obj_spex.sc_knn, fp)
# utils.save_object(obj_spex, f'{outDir}/obj_spex.pkl')


[2024-08-01 15:58:50,410/DEBUG] Term1 (original) start
[2024-08-01 15:59:33,567/DEBUG] Term1 (torch) start
[2024-08-01 15:59:38,982/DEBUG] First-term calculation done!
> /home/notify/Documents/2406-work/SpaMint/spamint/gd_torch.py(92)run_gradient()
     90         pdb.set_trace()
     91         # self.term2_df, self.loss2 = self.term2()
---> 92         '''
     93 
     94 

*** AttributeError: GradientDescent has no attr loss1_. Did you mean: 'loss1'?


In [ ]:
from spamint import optimizers
importlib.reload(optimizers)
import cProfile
'''
print(optimizers.findCellKNN(obj_spex.st_coord, obj_spex.st_tp, 
                            obj_spex.sc_agg_meta,
                            obj_spex.gradient_descent_solver.sc_coord,
                            obj_spex.gradient_descent_solver.K ))
'''
self=obj_spex.gradient_descent_solver

logger.debug("calcNeighborAffMat")
#cProfile.runctx('''
optimizers.calcNeighborAffinityMat(self.spots_nn_lst, self.spot_cell_dict, self.lr_df, self.alter_sc_exp)
#                ''', globals(), locals(), sort='tottime')
logger.debug("OK")

logger.debug("calcAffMat")
optimizers.calculate_affinity_mat(self.lr_df, self.alter_sc_exp)
logger.debug("ok")

[2024-07-09 00:34:47,068/DEBUG] calcNeighborAffMat
[2024-07-09 00:35:04,701/DEBUG] OK
[2024-07-09 00:35:04,708/DEBUG] calcAffMat
[2024-07-09 00:35:18,771/DEBUG] ok
